In [1]:
import numpy as np

def parse_board(board):
    return np.array([[int(c) if c.isdigit() else 0 for c in row.strip() if c != " "] for row in board.strip().split('\n')])

def print_board(board):
    for i, row in enumerate(board):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - -")
        for j, val in enumerate(row):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            if j == 8:
                print(val)
            else:
                print(str(val) + " ", end="")
                print
    print()

def board_is_solved(board):

    for num in range(1, 10):
        # Check rows
        for i in range(9):
            count = 0
            for j in range(9): #cycle through columns of a fixed row
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False
                    
        # check columns
        for j in range(9):
            count = 0
            for i in range(9): #cycle through rows of a fixed column
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False

    # Check boxes
        for box_x in range(1, 3):
            for box_y in range(1, 3):
                count = 0
                for i in range(box_y * 3, box_y * 3 + 3):
                    for j in range(box_x * 3, box_x * 3 + 3):
                        if board[i][j] == num:
                            count += 1
                        if(count > 1): #if a number appears more than one time in a box 3x3
                            return False
                if count < 1: #if a number doesn't appear in a box
                    return False
    return True

def board_is_full(board):
    
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return False
    return True

In [2]:
def get_first_free_pos(board):
    
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return (i,j) #row, column
    
    return None

def get_valid_num_in_position(board, pos):
    
    list = [1,2,3,4,5,6,7,8,9]
    #check row
    for j in range(9):
        if board[pos[0], j] in list:
            list.remove(board[pos[0], j])
            
    #check column
    for i in range(9):
        if board[i, pos[1]] in list:
            list.remove(board[i, pos[1]])
            
    #check box 3x3
    box_x = pos[1] // 3
    box_y = pos[0] // 3
    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x*3, box_x*3 + 3):
            if board[i, j] in list:
                list.remove(board[i, j])
    
    return list


In [3]:
def solve_board(board):
    
    #print_board(board)
    
    if board_is_full(board):
        return board_is_solved(board)
    else:
        pos = get_first_free_pos(board)
        
    for num in get_valid_num_in_position(board, pos):
        board[pos[0]][pos[1]] = num
        
        if solve_board(board):
            return True
        
        board[pos[0]][pos[1]] = 0
    
    return False

In [38]:
# Example Board
sudoku_board = """
37. 5....6
... 36. .12
... .91 75.
... 154 .7.
..3 .7. 6..
.5. 638 ...
.64 98. ...
59. .26 ...
2.. ..5 .64
"""

board = parse_board(sudoku_board)
print("Example Sudoku:")
print_board(board)


if(solve_board(board)):
    print("Soluzione: ")
    print_board(board)
else:
    print("Board non risolvibile")


Example Sudoku:
3 7 0 | 5 0 0 | 0 0 6
0 0 0 | 3 6 0 | 0 1 2
0 0 0 | 0 9 1 | 7 5 0
- - - - - - - - - - - -
0 0 0 | 1 5 4 | 0 7 0
0 0 3 | 0 7 0 | 6 0 0
0 5 0 | 6 3 8 | 0 0 0
- - - - - - - - - - - -
0 6 4 | 9 8 0 | 0 0 0
5 9 0 | 0 2 6 | 0 0 0
2 0 0 | 0 0 5 | 0 6 4

Soluzione: 
3 7 1 | 5 4 2 | 8 9 6
9 8 5 | 3 6 7 | 4 1 2
6 4 2 | 8 9 1 | 7 5 3
- - - - - - - - - - - -
8 2 6 | 1 5 4 | 3 7 9
4 1 3 | 2 7 9 | 6 8 5
7 5 9 | 6 3 8 | 2 4 1
- - - - - - - - - - - -
1 6 4 | 9 8 3 | 5 2 7
5 9 7 | 4 2 6 | 1 3 8
2 3 8 | 7 1 5 | 9 6 4



In [4]:
# difficult sudoku
sudoku = """
8........
..36.....
.7..9.2..
.5...7...
....45.7.
...1...3.
..1....68
..85...1.
.9....4..
"""

board = parse_board(sudoku)
print("Example Sudoku:")
print_board(board)


if(solve_board(board)):
    print("Soluzione: ")
    print_board(board)
else:
    print("Board non risolvibile")

Example Sudoku:
8 0 0 | 0 0 0 | 0 0 0
0 0 3 | 6 0 0 | 0 0 0
0 7 0 | 0 9 0 | 2 0 0
- - - - - - - - - - - -
0 5 0 | 0 0 7 | 0 0 0
0 0 0 | 0 4 5 | 0 7 0
0 0 0 | 1 0 0 | 0 3 0
- - - - - - - - - - - -
0 0 1 | 0 0 0 | 0 6 8
0 0 8 | 5 0 0 | 0 1 0
0 9 0 | 0 0 0 | 4 0 0

Soluzione: 
8 1 2 | 7 5 3 | 6 4 9
9 4 3 | 6 2 8 | 7 5 1
6 7 5 | 4 9 1 | 2 8 3
- - - - - - - - - - - -
2 5 6 | 3 8 7 | 1 9 4
1 3 9 | 2 4 5 | 8 7 6
7 8 4 | 1 6 9 | 5 3 2
- - - - - - - - - - - -
5 2 1 | 9 7 4 | 3 6 8
4 6 8 | 5 3 2 | 9 1 7
3 9 7 | 8 1 6 | 4 2 5

